## Daten Laden

In [ ]:
file_path = "BIG_2025-05-09__22_03_20-normalized-supplemented.csv"
print(file_path)

In [ ]:
import pandas as pd

df = pd.read_csv(file_path)

# Drop command column
df = df.drop(columns=['command'])

# Display the first few rows to understand the structure
df.head()

## Korrelationsmatrix

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='RdYlGn')

## 1. Versuch (Regression)

In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Zielvariable: Annahme, dass die erste Spalte die zu schätzende Zahl enthält
y = df.iloc[:, 0]

# Feature, das immer genutzt werden soll: 'instructions'
X_instructions = df[['instructions:u']]

# Potenzielle zusätzliche Features: Alle Spalten außer der Zielvariable und 'instructions'
potential_features = df.drop(columns=[df.columns[0], 'instructions:u'])

# Automatische Auswahl der 4 besten Features basierend auf f_regression
selector = SelectKBest(score_func=f_regression, k=4)
X_selected = selector.fit_transform(potential_features, y)

# Ausgabe der Namen der ausgewählten Features
selected_feature_names = potential_features.columns[selector.get_support()]
print("Ausgewählte Features:", selected_feature_names.tolist())

# Falls gewünscht: Umwandeln in DataFrame, um die Spaltennamen beizubehalten
X_selected_df = pd.DataFrame(X_selected, columns=selected_feature_names, index=df.index)

# Kombination der 'instructions'-Spalte mit den ausgewählten Features
X_combined = pd.concat([X_instructions, X_selected_df], axis=1)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Training eines Regressionsmodells (z. B. Lineare Regression)
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen und Evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))


## 2. Versuch (Regression)

### Modell trainieren

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Zielvariable: erste Spalte (angenommen, es handelt sich um numerische Werte)
y = df.iloc[:, 0]

# Fixes Feature: 'instructions'
X_instructions = df[['instructions:u']]

# Kandidaten für die automatische Feature-Auswahl:
# Entfernen der Zielvariable und der Spalte 'instructions'
X_candidates = df.drop(columns=[df.columns[0], 'instructions:u'])

# Basisregressor für die Feature-Selektion (hier: Lineare Regression)
model = LinearRegression()

# Sequential Feature Selector (forward selection) wählt 4 zusätzliche Features aus
sfs = SequentialFeatureSelector(
    model,
    n_features_to_select=4,
    direction='forward',
    scoring='neg_mean_squared_error',
    cv=5  # 5-fache Kreuzvalidierung
)

# Durchführung der Feature-Selektion auf den Kandidatenfeatures
sfs.fit(X_candidates, y)

# Ermitteln der Namen der ausgewählten Features
selected_feature_names = X_candidates.columns[sfs.get_support()]
print("Ausgewählte Features:", selected_feature_names.tolist())

# Zusammenführen des fixen Features 'instructions' mit den ausgewählten Features
X_selected = pd.concat([X_instructions, X_candidates[selected_feature_names]], axis=1)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Modelltraining auf den kombinierten Features
model.fit(X_train, y_train)

# Vorhersagen und Evaluation
y_pred = model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))

# Ausgabe der Regressionsparameter
print("\nRegression Parameter:")
print("Achsenabschnitt (Intercept):", model.intercept_)

# Ausgabe der Koeffizienten mit den zugehörigen Feature-Namen
print("Koeffizienten:")
for feature, coef in zip(X_selected.columns, model.coef_):
    print(f"{feature}: {coef}")

### Modell testen

In [ ]:
def do_regression(instructions, dTLB_loads, cache_misses, cstate_core_c6_residency, unc_m_clockticks):
  return -417.5947581136508 + instructions * 4.65216312028892e-11 + dTLB_loads * 7.883112920541098e-10 + cache_misses * -1.9612014211040394e-07 + cstate_core_c6_residency * 4.18166881063541e-09 + unc_m_clockticks * 1.3126082256029556e-07

# sleep 10 (expected 2.125)
print("sleep 10 (expected 2.125)")
print(do_regression(168777.863665457, 41575.009771487, 1361.36887244182, 27904585074.6458, 177728131.709414))

print("")

# sum_up_benchmark (expected 14.505)
print("sum_up_benchmark (expected 14.505)")
print(do_regression(11664606808.4586, 2175275696.69778, 6728517.43140718, 28062502860.874, 2390791162.75018))

print("")

# stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 16.39)
print("stress-ng --cpu 2 --cpu-method all --timeout 10s (expected 16.39)")
print(do_regression(15248513772.5695, 2013098946.16777, 56550.6387563183, 27893525904.2613, 2389444709.36587))

## 3. Versuch (Schwierig auf Colab...)

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class CustomSelector(BaseEstimator, TransformerMixin):
    """
    Transformer, der die Spalte 'instructions' immer mit einbezieht
    und zusätzlich eine Liste von weiteren Features auswählt, die als Parameter übergeben wird.
    """
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Es werden immer 'instructions' plus die angegebenen features zurückgegeben
        return X[['instructions:u'] + list(self.features)]

def optimal_feature_selection_regression_cv(df, test_size=0.2, random_state=42):
    """
    Verwendet GridSearchCV, um die optimale Feature-Kombination zu finden.
    'instructions' wird stets genutzt und zusätzlich werden 4 weitere Features
    aus den Kandidaten (alle außer der Zielvariable und 'instructions') ausgewählt.

    Rückgabe:
      dict: Mit den besten zusätzlichen Features, dem CV-Score,
            Test-MSE, Achsenabschnitt und Koeffizienten.
    """
    # Zielvariable: erste Spalte
    y = df.iloc[:, 0]
    # Prädiktoren: alle Spalten außer der Zielvariable
    X = df.drop(columns=[df.columns[0]])

    # Kandidaten für zusätzliche Features: alle außer 'instructions'
    candidates = list(X.drop(columns=['instructions:u']).columns)

    # Pipeline: CustomSelector + LinearRegression
    pipeline = Pipeline([
        ('selector', CustomSelector(features=[])),
        ('regressor', LinearRegression())
    ])

    # Erzeuge den Parametergrid: alle Kombinationen von 4 Features aus den Kandidaten
    param_grid = {
        'selector__features': [list(comb) for comb in combinations(candidates, 4)]
    }

    grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X, y)

    best_features = grid.best_params_['selector__features']
    best_score = grid.best_score_

    print("Beste zusätzliche Features:", best_features)
    print("Bester CV Score (neg_mean_squared_error):", best_score)

    # Aufteilen in Trainings- und Testdaten
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Bestes Modell trainieren
    best_model = grid.best_estimator_
    best_model.fit(X_train, y_train)

    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print("Finales Test-MSE:", mse)

    # Regressionsparameter ausgeben
    regressor = best_model.named_steps['regressor']
    print("\nRegression Parameter:")
    print("Achsenabschnitt (Intercept):", regressor.intercept_)

    coef_dict = {}
    # Die verwendeten Spalten: 'instructions' plus die best_features
    features_used = ['instructions:u'] + best_features
    print("Koeffizienten:")
    for feature, coef in zip(features_used, regressor.coef_):
        print(f"{feature}: {coef}")
        coef_dict[feature] = coef

    return {
        "best_features": best_features,
        "cv_score": best_score,
        "test_mse": mse,
        "intercept": regressor.intercept_,
        "coefficients": coef_dict
    }

# Beispiel zur Nutzung:
# df = pd.read_csv('data.csv')
# result = optimal_feature_selection_regression_cv(df)
# print(result)

optimal_feature_selection_regression_cv(df)